# EEG Cropping and Trial assignment
Status: Implemented
 -- Fully automatized

In [1]:
from tqdm.notebook import tqdm
from utils.utils import *

In [2]:
# get all files from participants
folders = get_participant_folder_per_participant()
eeg_file_groups = get_eeg_files_per_participant()
assert (set(folders.keys()) == set(eeg_file_groups.keys()))

### Assign trials to annotations

In [3]:
# Iterate over all participants
for participant in tqdm(folders):
    print('----------------------------------------------')
    print(PARTICIPANT, participant)

    print('(01/07) Check artifact reasoning file')
    # check whether reasoning file was filled out correctly
    artifact_reasoning = pd.read_csv(
        get_manual_ICA_reasoning_path(participant, True), sep=SEPARATOR, dtype={PARTICIPANT: str})
    check_manual_ICA_reasoning(artifact_reasoning)

    print('(02/07) anonymize prepared EEG data')
    eeg_path = get_prepared_eeg_path(participant)
    if not eeg_path.exists():
        print(f'\tNo prepared EEG file {eeg_path.as_posix()} found.')
        continue
    anonymize_eeg_data({
        EEG_FILE_HEADER_ENDING: eeg_path.with_suffix(EEG_FILE_HEADER_ENDING),
        EEG_FILE_MARKER_ENDING: eeg_path.with_suffix(EEG_FILE_MARKER_ENDING),
        EEG_FILE_DATA_ENDING: eeg_path.with_suffix(EEG_FILE_DATA_ENDING)})
    # eeg_path = eeg_file_groups[participant][EEG_FILE_HEADER_ENDING]

    print('(03/07) Load prepared EEG data')
    eeg_data, sfreq = load_eeg_data(eeg_path, preload=True)

    print('(04/07) Crop data to useful part')
    # crop to section containing content, to ignore setup or finishing (especially muscular movement)
    crop_to_complete_annotation_range(eeg_data)  # cannot crop EEG in-between
    print('info', eeg_data.info, eeg_data.n_times,
          eeg_data.first_time, eeg_data.first_samp)

    print('(05/07) Save EEG data')
    # save cropped EEG data in new file
    export_eeg_brainvision(eeg_data, get_cropped_eeg_path(participant))

    print('(06/07) Load behavioral events prepared for EEG synchronization')
    behavioral_events = pd.read_csv(
        get_behavioral_events_path(participant), sep=SEPARATOR, dtype={PARTICIPANT: str})

    print('(07/07) Assign trial to annotations')
    # somehow get eeg timestamp / order to match eeg section to snippet (behavioral or via condition files?)
    automatic_success, data = assign_trials_to_annotations(
        eeg_data, behavioral_events)
    print('automatic success' if automatic_success else '\tno automatic success, please mark the correct snippet trial for each annotation.')
    data.to_csv(get_trial_annotations_path(
        participant, performed=automatic_success), sep=SEPARATOR, index=False)

  0%|          | 0/24 [00:00<?, ?it/s]

----------------------------------------------
Participant 003
(01/07) Check artifact reasoning file
(02/07) anonymize prepared EEG data
(03/07) Load prepared EEG data
Extracting parameters from ..\08-Data-Trial_Recordings\processed\..\prepared_EEG_files\EEG\AoCfrp_003_New Reference.vhdr...
Setting channel info structure...


e:\PHD\Studies\aoc-frp-main-studies\Main_Study_Part\09-Task-Evaluate_Data\utils\eeg_helpers.py:131: RuntimeWarning: No coordinate information found for channels ['A2']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw_eeg = mne.io.read_raw_brainvision(filepath, eog=tuple(EOG_CHANNELS),


Reading 0 ... 1314489  =      0.000 ...  2628.978 secs...


e:\PHD\Studies\aoc-frp-main-studies\Main_Study_Part\09-Task-Evaluate_Data\utils\eeg_helpers.py:131: RuntimeWarning: Not setting positions of 5 eog/misc channels found in montage:
['A2', 'VEOGP', 'VEOGM', 'HEOGP', 'HEOGM']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw_eeg = mne.io.read_raw_brainvision(filepath, eog=tuple(EOG_CHANNELS),


...

(06/07) Load behavioral events prepared for EEG synchronization
(07/07) Assign trial to annotations
annotations: 217 behavioral_events: 217
automatic success
----------------------------------------------
Participant 106
(01/07) Check artifact reasoning file
(02/07) anonymize prepared EEG data
(03/07) Load prepared EEG data
Extracting parameters from ..\08-Data-Trial_Recordings\processed\..\prepared_EEG_files\EEG\AoCfrp_106_New Reference.vhdr...
Setting channel info structure...
Reading 0 ... 2017689  =      0.000 ...  4035.378 secs...


e:\PHD\Studies\aoc-frp-main-studies\Main_Study_Part\09-Task-Evaluate_Data\utils\eeg_helpers.py:131: RuntimeWarning: No coordinate information found for channels ['A2']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw_eeg = mne.io.read_raw_brainvision(filepath, eog=tuple(EOG_CHANNELS),
e:\PHD\Studies\aoc-frp-main-studies\Main_Study_Part\09-Task-Evaluate_Data\utils\eeg_helpers.py:131: RuntimeWarning: Not setting positions of 5 eog/misc channels found in montage:
['A2', 'VEOGP', 'VEOGM', 'HEOGP', 'HEOGM']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw_eeg = mne.io.read_raw_brainvision(filepath, eog=tuple(EOG_CHANNELS),


(04/07) Crop data to useful part
info <Info | 8 non-empty values
 bads: []
 ch_names: Fp1, Fp2, F7, F3, Fz, F4, F8, FC5, FC3, FCz, FC4, FC6, T7, C3, ...
 chs: 27 EEG, 1 misc, 4 EOG
 custom_ref_applied: False
 dig: 30 items (3 Cardinal, 27 EEG)
 highpass: 0.0 Hz
 lowpass: 250.0 Hz
 meas_date: unspecified
 nchan: 32
 projs: []
 sfreq: 500.0 Hz
> 1936216 89.842 44921
(05/07) Save EEG data


Python\Python311\site-packages\pybv\io.py:690: UserWarning: Encountered unsupported non-voltage units: n/a
Note that the BrainVision format specification supports only µV.
  warn(msg)


(06/07) Load behavioral events prepared for EEG synchronization
(07/07) Assign trial to annotations
annotations: 223 behavioral_events: 223
automatic success
